### Active Adjustment

策略：根据现价主动调整头寸。选定一个tick长度，每次都在现价附近选定固定长度的tick。

比如我们选3为固定长度。

1. 首次提供：

    在当前价格附近选择三个tick，提供流动性。

2. 调整：

    · 当当前价格超出了选定的tick，所有的币会被转换成价格降低的那种币。取出流动性，将其中一半的token换成另一种，并在当前价格附近重新提供流动性。

    · 为了简化，我们不把收取的手续费作为头寸添加到池子中

    · 这种高频调整的策略下，取出流动性和重新添加流动性以及swap的gas fee将是很大的一块成本，我看了一下etherscan上的记录，就先都拿20美元一次来算。所以每一次调整都会花费20 * 3 usdc（如果在bnb上手续费比较低，结果可能会有所不同）

In [42]:
import sys
sys.path.append("..") 
import pandas as pd
from datetime import datetime
from importlib import reload
from poolData import swapData
from utils import utils
reload(swapData)
reload(utils)

query = swapData.SwapDataQuery()
utils = utils.utils()

In [3]:
# position info
# 选定池子： token0: usdc(decimal: 6), token1: weth(decimal: 18)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
query.query_positions(pool_id=pool_id, block_gte=16859000, limit=10, orderBy="owner")
decimal0 = 6
decimal1 = 18
# 初始头寸的美元价值
initial_position = 10000

# 选定时间
begin = datetime(2023, 3, 19)
end = datetime.now()

pool_id: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 - type: <class 'str'>


In [4]:
# 获取起始时间和结束时间的价格
# 假设本金是1000刀，token0是usdc，数量是500
token0_amount = 5000
# 计算token1数量的时候需要知道价格
# 获取价格
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640")
liquidity_data
price_begin = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][0]))
price_end = utils.sqrtPrice2Price(int(liquidity_data["sqrtPrice"][liquidity_data.shape[0]-1]))
price_begin_real = utils.price2RealPrice(price_begin, decimal0=decimal0, decimal1=decimal1)
price_end_real = utils.price2RealPrice(price_end, decimal0=decimal0, decimal1=decimal1)
# 拿到价格之后才可以计算初始头寸
token1_amount = 5000 * price_begin_real

# 计算价格范围
tickLower = -887270
tickUpper = 887270
price_low = utils.tickIndex2Price(tickLower)
price_upper = utils.tickIndex2Price(tickUpper)
print("this is the lower price: ", price_low)
print("this is the upper price: ", price_upper)


this is the lower price:  2.939544628365392e-39
this is the upper price:  3.4018874568203963e+38


In [5]:
# 计算liquidity 
# liquidity的计算会比较复杂，因为liquidity是会随着价格变化而实时调整的。所以我们需要算一个liquidity的时间序列出来。
# 首先我们需要一个当前价格的时间序列, 根据当前的价格计算出当前的tick，然后根据tick space来择距离当前tick最近的一个tick
liquidity_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)
# l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
# l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
# l = min(l1, l2)
# print("l1:", l1)
# print("l2:", l2)
# print("l:", l)

In [6]:
# 计算出当前的tick
# 把sqrtPrice转为index
liquidity_data["sqrtPrice"] = liquidity_data["sqrtPrice"].astype(float)
liquidity_data["real_price"] = liquidity_data["sqrtPrice"].astype(float).apply(lambda x: utils.sqrtPrice2Price(x))
liquidity_data["currentTickIndex"] = liquidity_data["real_price"].astype(float).apply(lambda x: utils.price2TickIndex(x))


In [7]:
# 根据tick spacing 选择距离当前价格最近的一个tick
# 不同费率的池子有不同的tick spacing
tick0 = 10 # 0.05% fee
tick1 = 60 # 0.3% fee
tick3 = 200 # 1.0% fee
# 计算出最近的tick index
liquidity_data["nearestTick"] = liquidity_data["currentTickIndex"].apply(lambda x: utils.nearestTick(x, tick0))

### 计算动态调整的liquidity

如何计算liquidity？

随着价格的变化，持有的token的美元价值以及两种token的持有比例会大幅度变动。

但为了简化，在调整头寸的时候我们假设每次的价值都为初始头寸的价值，并且收集到的手续费不在调整的时候成为流动性。

**todo** 头寸token的美元价值动态调整

已知upper tick，lower tick，position的总价值，需要计算liquidity。

计算liquidity的公式：

```python
l1 = (token0_amount * (price_low ** 0.5) * (price_upper ** 0.5)) / ((price_upper ** 0.5) - (price_low ** 0.5))
l2 = token1_amount / ((price_upper ** 0.5) - (price_low ** 0.5))
l = min(l1, l2)
```
其中，token0_amount为5000，token1_amount为5000/price_token1

In [9]:
liquidity_data

,periodStartUnix,liquidity,sqrtPrice,token0Price,token1Price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,tvlUSD,volumeToken0,...,feesUSD,txCount,open,high,low,close,real_price,currentTickIndex,nearestTick,position_token0_amount
0,1679158800,31691854789759819627,1.871667e+33,1791.851542392230339967547914533666,0.0005580819483878331967604985081167192,201410,2314975469199928457298700227744982,1111162292302232016625192018264273618935405,362517939.1226720089381000671278629,0,...,0,372,1798.390894813620679474180985197866,1803.236399827628569085607995221095,1769.953587075568576406795835267567,1791.851542392230339967547914533666,5.580819e+08,201410.233545,201410.0,5000
1,1679162400,32915185914729732691,1.867893e+33,1799.098710137099104209582500255936,0.0005558338707962253356429190522474716,201369,2315038215475779131181192686558395,1111189202831556094328153678234461295869361,363811618.6075028876192885997227013,0,...,0,366,1790.640320160079804025064362917098,1799.108571408040728063168297875355,1786.686354081499257894742581324593,1799.098710137099104209582500255936,5.558339e+08,201369.867963,201370.0,5000
2,1679166000,34369179026466248085,1.869483e+33,1796.040997899866322473028892703993,0.0005567801632419932350116030494839081,201386,2315126854003039415933596971303595,1111241826769754789231620166343530092470985,363967492.8611761592527996744286012,0,...,0,333,1799.099225334904656531075427168004,1809.622901299084675474781496124223,1794.120699231514675114167039962852,1796.040997899866322473028892703993,5.567802e+08,201386.879073,201390.0,5000
3,1679169600,34183163223601833767,1.868781e+33,1797.389578204559120182790388692096,0.000556362411424970994583196746389777,201379,2315168855482994101747218874079807,1111263525863761589158339866419647898430929,363205815.9317762712257394030005428,0,...,0,317,1796.054690522833853305120372567828,1805.57781837591635090288239389413,1792.725482994450340228488201862034,1797.389578204559120182790388692096,5.563624e+08,201379.372887,201380.0,5000
4,1679173200,34246952167731560476,1.875753e+33,1784.054320332921814271944303424757,0.0005605210495011106585938907612211571,201453,2315193497548537858877306471738285,1111292420021385719852093400706179840724576,360909125.4200933323216227809336806,0,...,0,309,1797.199358810234823401563218695319,1797.220201287329445413990798860576,1783.038903387140753714413070004309,1784.054320332921814271944303424757,5.605210e+08,201453.845567,201450.0,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,1681790400,25533083250979595537,1.735475e+33,2084.117582639578069480006044906994,0.0004798193769535206815356005851068624,199899,2347528144524911754928457617179668,1128482450993326625278036835283134896854505,489434049.9675288030271429817686929,0,...,0,218,2081.497694487869687222682664093655,2084.529675957757056077910305447468,2080.386511319579054405466605688624,2084.117582639578069480006044906994,4.798194e+08,199899.197224,199900.0,5000
732,1681794000,36725047966336051552,1.733573e+33,2088.693263494337118490042784007292,0.0004787682411188622133554301775131516,199877,2347554039965560457767323894900170,1128490767099911908556137642501683077590838,490999597.8439751093622808945250325,0,...,0,257,2084.11486713184889928003267926646,2091.747430453318572069574532630981,2084.109021188512922290667829372682,2088.693263494337118490042784007292,4.787682e+08,199877.265190,199880.0,5000
733,1681797600,35841261167438473165,1.726935e+33,2104.783267341858332165721751886721,0.0004751082999927613302174656730426363,199800,2347610548209729630724338560726687,1128503403938417507886547133690975048217956,492802561.4640636354274648785511676,0,...,0,315,2090.294694230760411117200272415645,2105.906950138221468086297906978458,2090.294694230760411117200272415645,2104.783267341858332165721751886721,4.751083e+08,199800.522720,199800.0,5000
734,1681801200,39401036249240560888,1.727983e+33,2102.229223844090413298796324178929,0.0004756855192848198806656127611403217,199812,2347626453

In [35]:
# 策略是在当前tick下提供流动性，选取3为长度好了，tick范围是[currentTick-3, currentTick+3]
# 初始头寸的美元价值为10000usdc
initial_position = 10000

liquidity_data["position_token0_amount"] = 5000.0
liquidity_data["real_price_token1"] = 1 / (utils.price2RealPrice(liquidity_data["real_price"], 6, 18))
liquidity_data["position_token1_amount"] = 5000  / liquidity_data["real_price_token1"]
liquidity_data["upperTickIndex"] = liquidity_data["currentTickIndex"] + 3
liquidity_data["lowerTickIndex"] = liquidity_data["currentTickIndex"] - 3

In [51]:
liquidity_data["upperPrice"] = liquidity_data["upperTickIndex"].apply(lambda x: utils.tickIndex2Price(x))
liquidity_data["lowerPrice"] = liquidity_data["lowerTickIndex"].apply(lambda x: utils.tickIndex2Price(x))
liquidity_data["liquidity"] = liquidity_data["liquidity"].astype(float)
liquidity_data["position_liquidity"] = utils.calculateLiquidity(liquidity_data["position_token0_amount"], liquidity_data["position_token1_amount"], liquidity_data["lowerPrice"], liquidity_data["upperPrice"])
liquidity_data["ratio"] = liquidity_data["position_liquidity"] / liquidity_data["liquidity"]

### 计算fee

现在有了liquidity的时间序列数据，可以计算头寸的手续费了。

In [52]:
# 获取所有swaps
swaps = query.query_swaps(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)

Error: Failed to get entities from store: canceling statement due to conflict with recovery, query = /* controller='filter',application='sgd217942',route='91044f11ec7c3947-484e4af45a237306',action='17072762' */
select 'Swap' as entity, to_jsonb(c.*) as data from (select  c.*
  from "sgd217942"."swap" c
 where c.block_range @> $1 and (c."id" > $2 and c."pool" = $3 and c."sqrt_price_x96" >= $4::numeric and c."sqrt_price_x96" <= $5::numeric and c."timestamp" > $6::numeric and c."timestamp" < $7::numeric)

 order by "id", block_range
 limit 1000) c -- binds: [17072762, "0x1ee31504489e20c47dc612440d50a17ab4a5f4d1cfa6a107ca023e3be5dd87e4#4711058", "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "0", "6277101735386680763835789423207666416102355444464034512896", "1679155200", "1681808334"]


In [53]:
# 为了简化 假设主动调整头寸的操作足够及时，每一次swap都在我们的头寸之内，所以swaps不需要筛选
liquidity_data = liquidity_data[["periodStartUnix", "ratio"]]
liquidity_data = liquidity_data.rename(columns={"periodStartUnix": "timestamp", "ratio": "ratio"})
# 先排序
swaps["timestamp"] = swaps["timestamp"].astype(int)
swaps = swaps.sort_values(by="timestamp")
liquidity_data = liquidity_data.sort_values(by="timestamp")
# 找liquidity data只时间最接近的来合并
merged_data = pd.merge_asof(
    swaps,
    liquidity_data[["timestamp", "ratio"]],
    on="timestamp",
    direction="nearest"
)
merged_data.head(3)

,id,timestamp,sender,recipient,origin,amount0,amount1,amountUSD,sqrtPriceX96,tick,logIndex,ratio
0,0x0861d00acafcd74202e82741c031db7417fbca4c45fe...,1679155223,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,0xe63df014b8481ecc14582ff09e88809b0ad4e5a9,-17615.76052,9.835103915662891668,17613.78905750513568184175674883613,1871596376831572468181234428468238,201409,17,1.242430e-20
1,0x060ca03d50d79c0196c3b46dcbebb3dfef605b4f815e...,1679155343,0xe592427a0aece92de3edee1f18e0157c05861564,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xbff9a1b539516f9e20c7b621163e676949959a66,5994.484682,-3.349237752536633017,5995.99888113471748052423946471036,1873201169334335818044136888319180,201426,168,1.242430e-20
2,0x00a4333a3aad92b9579333e786936077d987a2022a6f...,1679155355,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0xa62a7a20c38fd87d0ba3f06bfc84a3bce7596120,619.1124,-0.345987404437934662,619.337728969297592786277072330285,1873412840676047030438895530061802,201428,299,1.242430e-20


In [54]:
# 计算手续费
liquidity_data["ratio"] = liquidity_data["ratio"].astype(float)
merged_data['amount0'] = merged_data['amount0'].astype(float).apply(lambda x: max(0, x))
merged_data['amount1'] = merged_data['amount1'].astype(float).apply(lambda x: max(0, x))

merged_data["fee0"] = merged_data["amount0"] * merged_data['ratio'] * 0.0005
fee0_sum = merged_data["fee0"].sum()

merged_data["fee1"] = merged_data["amount1"] * merged_data['ratio'] * 0.0005
fee1_sum = merged_data["fee1"].sum()

print(fee0_sum)
print(fee1_sum)


4.770470435064525e-15
2.673554961787187e-18
